In [3]:
from scipy.stats import norm
import numpy as np
import yfinance as yf
import pandas as pd
from ta.volatility import AverageTrueRange
from datetime import datetime


def get_historical_data(stk):
    df = yf.download(stk,period='max', interval="1wk").dropna()
    df['year']=df.index.isocalendar().year.astype(int)
    return df


def trade_parameters(df, yield_perc):
    df["strike_price"] = df.Open.round(0)
    df["straddle_yield"] = yield_perc
    df["straddle_premium"] = df.strike_price * yield_perc
    
    df['ATR']=AverageTrueRange(high=df.High,low=df.Low,close=df.Close,window=5,fillna=False).average_true_range().rank(pct=True).shift(1)
    df['abs_change']=abs(df.Close-df.Open)
    df['abs_change_pct']=df.abs_change/df.Open
    df["profit_loss"] = df.straddle_premium - df.abs_change
    df["worst"] = df.straddle_premium - df.abs_change
    df["log_return"] = np.log((df.strike_price + df.profit_loss) / df.strike_price)
    df['pos_ret']=np.where(df.log_return>0,df.log_return,0)
    df['neg_ret']=np.where(df.log_return<0,df.log_return,0)
    df = df.iloc[1: , ::]
    
    lim=0.3
    df=df[df.ATR<lim]
    
    return df

def annual_group(df):
    df = df.groupby(["year"]).agg(
        {"log_return": "sum",'pos_ret':'sum','neg_ret':'sum','profit_loss':'mean','worst':'min'}
    )
    df['sharpe']=df.pos_ret/abs(df.neg_ret)
    df['recovery_periods']=(-df.worst/df.profit_loss)
    df['recovery_periods']=df['recovery_periods'].mask(df['recovery_periods']<0,0)
    return(df)
    
    
atm_option_data = pd.read_csv("historical_straddle_data/option_data_2022-09-02_2022-08-29.csv")

#print(blank)
arr = []
limited=['UNG']
atm_option_data['yield_perc']=(atm_option_data.atm_call_last + atm_option_data.atm_put_last) / atm_option_data.strike
atm_option_data=atm_option_data[atm_option_data.symbol.isin(limited)]
for i,j in zip(atm_option_data.symbol,atm_option_data.yield_perc):
    
    stock = i
    yield_perc = j
    df = get_historical_data(stock)
    #print(df.tail)
    df = trade_parameters(df, yield_perc)
    #print(df.tail)
    #df.to_csv(stock + ".csv")
    df = annual_group(df)

    #df=df.iloc[::,[0,3,4]]
    #print(df)
    
    years=np.arange(df.index[0], 2023, 1, dtype=int)
    zeros=np.zeros((len(years),len(df.columns)),dtype=float)
    blank=pd.DataFrame(columns=df.columns.values,index=years,data=zeros)

    df=df.replace(float('inf'),500)
    print(df)
    lr_avg=(min(
        np.average(df.log_return),
        np.percentile(df.sharpe,50)
    ))

    sharpe_avg=(min(
        np.percentile(df.sharpe,50),
          np.average(df.sharpe)
    ))
    lr_check=len(df.log_return[df.log_return>0])/len(df.log_return)
    sharpe_check=len(df.sharpe[df.sharpe>2])/len(df.sharpe)
    recovery_check=len(df.recovery_periods[df.recovery_periods<7])/len(df.recovery_periods)
    lr_sum=np.sum(df.log_return)
    
    df1=blank.add(df,fill_value=0)
    periods_traded=len(df1.log_return[df1.log_return != 0])/len(df1.log_return)
    
    out=[stock,lr_avg,sharpe_avg,lr_sum,lr_check,sharpe_check,recovery_check,periods_traded]

    arr.append(out)
    #print(arr)
    df.to_csv('out.csv')
finaldf=pd.DataFrame(data=arr,columns=['stock','lr_avg','sharpe_avg','lr_sum','lr_check','sharpe_check','recovery_check','periods_traded'])
print(finaldf)
finaldf.to_csv('weekly_mass_bt.csv')

[*********************100%***********************]  1 of 1 completed
      log_return   pos_ret   neg_ret  profit_loss      worst      sharpe  \
year                                                                       
2007    0.202410  0.202410  0.000000    86.190935  36.440586  500.000000   
2016    0.153282  0.205101 -0.051818     0.575781  -1.037186    3.958071   
2017    1.582003  1.634039 -0.052037     1.107633  -0.750936   31.401752   
2018    1.829826  1.988299 -0.158473     1.068559  -2.392812   12.546631   
2019    1.812588  1.864184 -0.051597     0.911991  -0.663751   36.130048   
2020    1.045519  1.548024 -0.502505     0.266291  -0.733438    3.080613   
2021    1.717139  1.837517 -0.120378     0.473315  -0.700625   15.264542   
2022   -0.233574  0.062801 -0.296375    -0.556146  -1.943438    0.211896   

      recovery_periods  
year                    
2007          0.000000  
2016          1.801355  
2017          0.677965  
2018          2.239288  
2019          0.7278

/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_ind